In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip uninstall openpyxl
!pip install openpyxl

     |████████████████████████████████| 132 kB 4.1 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219291 sha256=06d1c7dad5ce11db1db07dface3e7fb542850504e9126505d9ce4d111a3ee93b
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
     |████████████████████████████████| 11.3 MB 4.1 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 4.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.metrics import r2_score
import itertools
import pickle5 as pickle
import datetime
from dateutil.relativedelta import relativedelta

### 학습한 모델 불러와서 중장기수주예측 저장

In [ ]:
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/수주분석테이블.pkl', "rb") as fh:
  수주분석테이블 = pickle.load(fh)

df = 수주분석테이블
df.reset_index(inplace=True)
df=df[['납기일자','제품명','판매수량']]
df.납기일자 = pd.to_datetime(df['납기일자'])
a = df.groupby('제품명')['판매수량'].sum().sort_values(ascending=False)
a.index

df_prod = pd.DataFrame(df.제품명.value_counts())
df_prod.rename(columns={'Unnamed: 0': '제품명' , 'PRODNAME':'COUNT'}, inplace=True)

df_pred_model_0 = pd.DataFrame()
for i in range(len(a.index)):
  df1 = df[df['제품명']==a.index[i]]
  df1 = pd.DataFrame(df1.groupby('납기일자')['판매수량'].sum())
  df1.reset_index(inplace=True)

  # 빈날짜 채우기
  start_date = df1.납기일자.min()
  end_date = df1.납기일자.max()

  currentdate = start_date
  date_index = []
  while currentdate <=(end_date):
    date_index.append(currentdate)
    currentdate=currentdate+datetime.timedelta(days=1)

  data_index = pd.DataFrame(date_index,columns=['납기일자'])
  data_index['납기일자'] = pd.to_datetime(data_index['납기일자'])

  df1 = pd.merge(data_index,df1,on='납기일자',how='left')
  df1.fillna(0,inplace=True)
  df1.set_index('납기일자',inplace=True)
  df_prod1 = df1.copy()

  from tensorflow.python.keras.models import load_model



  model_bilstm = load_model(f'/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/lstm_save_우리데이터/{a.index[i]}.h5')

  # std_date = datetime.today().date()-timedelta(days=1) # today -> 실제사용시

  
  std_date = datetime.datetime.strptime("2021-04-25", "%Y-%m-%d") # 훈련용 기준날짜 21/4/25 라고 가정    

  testdate_start = std_date - relativedelta(months=7)
  testdate_end = std_date - relativedelta(days=1)

  test_data = df_prod1[testdate_start:testdate_end+relativedelta(days=1)]  

  scaler = MinMaxScaler()
  test_scaled = scaler.fit_transform(test_data)

  # 인풋 데이터셋 만들기
  def create_dataset (X, look_back = 1):
      Xs = []

      for i in range(len(X)-look_back):
          v = X[i:i+look_back]
          Xs.append(v)
      return np.array(Xs)

  LOOK_BACK=30
  X = create_dataset(test_scaled,LOOK_BACK)

  def prediction(model):
      prediction = model.predict(X)
      prediction = scaler.inverse_transform(prediction)
      return prediction



  # 예측값 출력
  prediction_bilstm = np.ceil(prediction(model_bilstm))

  result = pd.DataFrame()
  preds = list(itertools.chain.from_iterable(prediction_bilstm))
  result['예측중량']= preds
        
  # 예측기간 날짜 인덱스 만들기
  start_date = std_date
  end_date = std_date + relativedelta(months=6)

  currentdate = start_date
  date_index = []
  while currentdate <=(end_date):
    date_index.append(currentdate)
    currentdate=currentdate+datetime.timedelta(days=1)

  data_index = pd.DataFrame(date_index,columns=['납기일자'])
  data_index['납기일자'] = pd.to_datetime(data_index['납기일자'])

  pred = pd.concat([data_index,result],axis=1)
  pred['제품명']=a.index[i]
  중장기수주예측 = df_pred_model_0.append(pred)
  df_pred_model_0 = 중장기수주예측
  중장기수주예측.fillna(0,inplace=True)

def to_zero(x):
  if x<0:
    return 0
  else:
    return x

중장기수주예측['예측중량'] = 중장기수주예측['예측중량'].apply(to_zero)
중장기수주예측['납기일자'] = pd.to_datetime(중장기수주예측['납기일자'])

중장기수주예측


,납기일자,예측중량,제품명
0,2021-04-05,38200.0,PEMA-HR1000
1,2021-04-06,0.0,PEMA-HR1000
2,2021-04-07,106890.0,PEMA-HR1000
3,2021-04-08,88140.0,PEMA-HR1000
4,2021-04-09,101064.0,PEMA-HR1000
...,...,...,...
179,2021-10-01,60.0,PEMA-SPR
180,2021-10-02,60.0,PEMA-SPR
181,2021-10-03,60.0,PEMA-SPR
182,2021-10-04,60.0,PEMA-SPR
